<a href="https://colab.research.google.com/github/sera0911/asia_ai_study/blob/main/New_MachinLearning/prj_01_news_category_classfication_04_model_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 전체 기사를 크롤링 하여 학습시킨 모델로 오늘 헤드라인의 카테고리를 예측하기

In [ ]:
! pip install konlpy

In [40]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
import pickle
from konlpy.tag import Okt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [41]:
pd.set_option('display.unicode.east_asian_width', True)  #줄맞춤

In [42]:
#토큰 불러오기

with open('/content/datasets/news_token.pickle', 'rb') as f:
    token = pickle.load(f)

In [43]:
#라벨인코더 불러오기

with open('/content/datasets/category_encoder.pickle', 'rb') as f:
    encoder = pickle.load(f)

category = encoder.classes_
print(category)

['Culture' 'Economic' 'IT' 'Politics' 'Social' 'World']


In [44]:
#오늘자 뉴스 헤드라인 데이터 불러오기
df_news_today = pd.read_csv('/content/datasets/naver_news_titles_210617_headline.csv',
                            index_col=0)

print(df_news_today.head())

                                                        title  category
0         합당 속도 공감대 속 주도권 기싸움 벌인               Politics
1     영상  이준석 안철수  전쟁 같은 합당 되지 않게            Politics
2   안철수 이준석 당 대표로서 첫 공식 회동 합당 논의           Politics
3  이준석 안철수  합당 의지 변함 없어   새 당명 엔 이견......  Politics
4   김기현 국민의힘 대표 국회 교섭단체 대표연설 LIVE ......    Politics


In [45]:
print(df_news_today.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 0 to 205
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     206 non-null    object
 1   category  206 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB
None


In [46]:
df = df_news_today.drop_duplicates(subset=['title'])  #중복 헤드라인 기사 제거하기
sum_dup = df.title.duplicated().sum()
print(sum_dup)

0


In [47]:
#데이터를 피쳐와 타켓으로 분류

X = df['title']
Y = df['category']

Y 전처리

X전처리

In [48]:
# for문을 사용해서 X전체 텍스트를 형태소 단위로 분리시키기 
okt = Okt()
for i in range(len(X)):
    X[i] = okt.morphs(X[i])
print(X)

0      [합당, 속도, 공감, 대, 속, 주도, 권, 기, 싸움, 벌인]...          
1      [영상, 이준석, 안철수, 전쟁, 같은, 합당, 되지, 않게]...          
2      [안철수, 이준석, 당, 대표, 로서, 첫, 공식, 회, 동, 합당, 논의]...
3      [이준석, 안철수, 합당, 의지, 변함, 없어, 새, 당, 명, 엔, 이견]...
4      [김기현, 국민, 의, 힘, 대표, 국회, 교섭단체, 대표, 연설, LIVE]...
                                     ...                                
201    [지금, 은, 과학, 알츠하이머, 치매, 진행, 늦출, 수, 있다]...      
202    [유용하, 기자, 의, 사이언스, 톡, 생쥐, 를, 빼고, 과학, 연구, 를, 말,...
203    [유용하, 기자, 의, 사이언스, 톡, 수, 포자, 되면, 뇌, 인지력, 떨어져요]...
204    [유용하, 기자, 의, 사이언스, 톡, 엄마, 가, 아이, 구, 하려, 위험, 무릅...
205    [유용하, 기자, 의, 사이언스, 톡, 무작위, 같지만, 당신, 행동, 엔, 패턴,...
Name: title, Length: 206, dtype: object


In [49]:
# 형태소로 잘라 낸 뒤, 의미 없는 조사, 접속사, 감탄사 같은 단어들은 제거하기 위해 이 단어들을 모아놓은 파일 가져오기

stopwords = pd.read_csv('/content/datasets/stopwords.csv')

In [50]:
#한 글자인 단어와, stopwords로 불필요한 단어 제거하기

for i in range(len(X)):  #i는 각각의 문장들
    result = []
    for j in range(len(X[i])):  #j는 각 문장 안 단어들
        if len(X[i][j]) > 1:  #한글자인 단어들은 if문 안에 못들어간다
            if X[i][j] not in list(stopwords['stopword']):  #그리고 stopwords에 없는 단어들만 들어간다
                result.append(X[i][j])  #통과된 텍스트를 빈 리스트에 넣어주기
    X[i] = ' '.join(result)  #문장 별 리스트 요소들을 join으로 이어준다
    
print(X)

0                        합당 속도 공감 주도 싸움 벌인
1               영상 이준석 안철수 전쟁 합당 되지 않게
2                    안철수 이준석 대표 공식 합당 논의
3                    이준석 안철수 합당 의지 변함 이견
4        김기현 국민 대표 국회 교섭단체 대표 연설 LIVE
                            ...                       
201                     과학 알츠하이머 치매 진행 늦출
202      유용하 기자 사이언스 생쥐 빼고 과학 연구 이유
203     유용하 기자 사이언스 포자 되면 인지력 떨어져요
204    유용하 기자 사이언스 엄마 하려 위험 무릅쓰 까닭
205       유용하 기자 사이언스 무작위 같지만 행동 패턴
Name: title, Length: 206, dtype: object


In [51]:
#토크나이징 작업하기(숫자로 라벨화 작업)

tokened_X = token.texts_to_sequences(X)  #texts_to_sequences = 단어를 시퀀스형태로 변환하기(문장변화는 여기서 일어난다)
print(tokened_X[0])  #첫줄만 확인하기

[877, 219, 1245, 576, 900, 9809]


In [52]:
#16개의 길이를 맞추기 위해 짧은 문장은 앞쪽에 0으로 채워준다

X_pad = pad_sequences(tokened_X, 27)  #pad_sequences= 27이 안되는 문장을 0으로 채워준다(앞쪽에)
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   877   219  1245
    576   900  9809]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0    41     1   211   385
    877  2291  3264]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   211     1    19
    451   877   168]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     1   211
    877  1491  4089]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   555     7    19   266  3733
     19  1139 15093]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     7   555    21  3733    19  1139  4114
   2672

In [53]:
#모델 불러오기

model = load_model('/content/news_classfication_0.7548006176948547.h5')

In [54]:
#모델을 이용해 예측하기

predict = model.predict(X_pad)
print(predict[0])  #첫줄을 예측

#출력결과 각 카테고리별로 정확도가 예측된다

[0.05539251 0.07450767 0.00727854 0.34097856 0.08456031 0.4372824 ]


In [55]:
#출력결과 3번째 인덱스가 9.77804363e-01으로 가장 크게 나온다 3번인덱스 = 정치 = 옳은 결과가 나옴
result = np.argmax(predict[0])
print(category[result])  #카테고리를 가져와서 정확도가 높은 결과가 나온 인덱스를 출력해준다

World


In [56]:
# predict에서 가장 값이 큰 인덱스만 가져오기

predict_category = []
for pred in predict:
    predict_category.append(category[np.argmax(pred)])

print(predict_category)

['World', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Culture', 'Politics', 'Politics', 'Social', 'Economic', 'Economic', 'Economic', 'Politics', 'Politics', 'Politics', 'Politics', 'Economic', 'Economic', 'Politics', 'Culture', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Culture', 'Economic', 'World', 'Politics', 'Politics', 'Politics', 'Politics', 'Politics', 'Social', 'Economic', 'Economic', 'Economic', 'Social', 'Economic', 'IT', 'Economic', 'Economic', 'IT', 'Culture', 'Culture', 'World', 'Economic', 'Economic', 'Economic', 'Economic', 'Economic', 'Economic', 'Economic', 'IT', 'Economic', 'Economic', 'Economic', 'IT', 'Social', 'Economic', 'Economic', 'Economic', 'Social', 'Social', 'Economic', 'Economic', 'Economic', 'Economic', 'Social', 'Social', 'Social', 'Social', 'Social', 'Social', 'Social', 'Social', 'Social', 'World', 'Social', 'Soc

In [57]:
print(Y)

0      Politics
1      Politics
2      Politics
3      Politics
4      Politics
         ...   
201          IT
202          IT
203          IT
204          IT
205          IT
Name: category, Length: 206, dtype: object


In [58]:
#컬럼 추가
df_news_today['predict'] = predict_category
print(df_news_today.head())

                                                        title  ...   predict
0         합당 속도 공감대 속 주도권 기싸움 벌인               ...     World
1     영상  이준석 안철수  전쟁 같은 합당 되지 않게            ...  Politics
2   안철수 이준석 당 대표로서 첫 공식 회동 합당 논의           ...  Politics
3  이준석 안철수  합당 의지 변함 없어   새 당명 엔 이견......  ...  Politics
4   김기현 국민의힘 대표 국회 교섭단체 대표연설 LIVE ......    ...  Politics

[5 rows x 3 columns]


In [59]:
#0으로 채워진 컬럼 추가
df_news_today['OX'] = 0
print(df_news_today.head())

                                                        title  ... OX
0         합당 속도 공감대 속 주도권 기싸움 벌인               ...  0
1     영상  이준석 안철수  전쟁 같은 합당 되지 않게            ...  0
2   안철수 이준석 당 대표로서 첫 공식 회동 합당 논의           ...  0
3  이준석 안철수  합당 의지 변함 없어   새 당명 엔 이견......  ...  0
4   김기현 국민의힘 대표 국회 교섭단체 대표연설 LIVE ......    ...  0

[5 rows x 4 columns]


In [60]:
#OX컬럼에 타이틀과 predict이 맞으면 o 아니면 x로 표기
for i in range(len(df_news_today.predict)):
    if df_news_today.category[i] == df_news_today.predict[i]:
        df_news_today.OX[i] = 'O'
    else:
        df_news_today.OX[i] = 'X'

print(df_news_today.head())

                                                        title  ... OX
0         합당 속도 공감대 속 주도권 기싸움 벌인               ...  X
1     영상  이준석 안철수  전쟁 같은 합당 되지 않게            ...  O
2   안철수 이준석 당 대표로서 첫 공식 회동 합당 논의           ...  O
3  이준석 안철수  합당 의지 변함 없어   새 당명 엔 이견......  ...  O
4   김기현 국민의힘 대표 국회 교섭단체 대표연설 LIVE ......    ...  O

[5 rows x 4 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [61]:
#예측 값을 알아보기
print(df_news_today.OX.value_counts() / len(df_news_today.OX))

O    0.529126
X    0.470874
Name: OX, dtype: float64


In [62]:
#내용 확인해보기

df_news_today.iloc[-30:, 0:3]

,title,category,predict
176,우주정거장 건설 선저우 12호 유인우주선 오늘 발사,World,World
177,중국 우주인 3명 자체 우주정거장에 보낸다 우주로 간다 ...,World,IT
178,도쿄 아파트값 연봉의 11배인데 서울은 언론도 놀랐다 정영효의 인사이드재팬...,World,Economic
179,한 대당 5000만원 넘지만 서 코로나에 인기 폭발한 정영효의 인사이드 재팬...,World,Social
180,스님 절반이 부업 잘나가던 불교 위기 맞은 까닭 정영효의 인사이드 재팬 ...,World,Economic
181,탈석탄사회 추진에 나무젓가락 가격 뛰는 이유 정영효의 인사이드 재팬 ...,World,Economic
182,산속 조난자 위치 소리 로 정확도 10배 높은 음원 위치 추적기술 나왔다...,IT,Social
183,살려주세요 산 속 조난자 위치 소리로 찾는다,IT,Culture
184,살려주세요 산속 조난사고 소리로 정확하게 찾아낸다...,IT,Social
185,살려주세요 비명 위치 실시간으로 찾는다,IT,Culture
